In [8]:
# Import necessary libraries
import os  
from dotenv import load_dotenv 
import sqlalchemy as db  
import pandas as pd  

# Set display options for pandas DataFrame
pd.set_option('display.max_rows', 500)  # Display all rows
pd.set_option('display.max_columns', 500)  # Display all columns
pd.set_option('display.width', 1000)  # Set display width for better readability

# Load environment variables from .env file
load_dotenv()

# Get database details from environment variables
db_username = os.getenv('DB_USERNAME')  
db_password = os.getenv('DB_PASSWORD') 
db_host = os.getenv('DB_HOST')  
db_port = os.getenv('DB_PORT')  
db_name = os.getenv('DB_NAME')  
db_schema = os.getenv('DB_SCHEMA')  


In [9]:
# Construct connection string using database details
connection_string = f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
# Create a SQLAlchemy engine object with the connection string
engine = db.create_engine(connection_string, echo=False)  # 'echo=False' suppresses logging

# Attempt to connect to the database
try:
    # Establish a connection to the database
    conn = engine.connect()
    # Print success message if connection is established
    print("Connection successful!")
except Exception as e:
    # Print error message if connection fails
    print("Connection failed:", e)


Connection successful!


In [10]:
# Create a metadata object to store database schema information
metadata = db.MetaData()  # Instantiate a MetaData object

# Reflect the 'customer' table from the database schema
customer = db.Table('customer', metadata, autoload=True, autoload_with=engine, schema=db_schema)
# Create a Table object named 'customer' by reflecting the existing table in the database

# Reflect metadata from the engine (database)
metadata.reflect(bind=engine)
# Reflect the metadata (tables, columns, etc.) from the database using the engine

# Print the metadata for the 'customer' table
print(repr(metadata.tables['dev.customer']))
# Print the representation of the 'customer' table metadata, including columns and constraints

# Print the keys (table names) of the reflected metadata
print(metadata.tables.keys())
# Print the keys (table names) of the metadata dictionary


Table('customer', MetaData(), Column('customerid', INTEGER(), table=<customer>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x00000219700698D0>, for_update=False)), Column('firstname', VARCHAR(length=50), table=<customer>), Column('lastname', VARCHAR(length=50), table=<customer>), Column('dateofbirth', DATE(), table=<customer>), Column('gender', CHAR(length=1), table=<customer>), Column('email', VARCHAR(length=100), table=<customer>), Column('phonenumber', VARCHAR(length=20), table=<customer>), Column('generateddate', DATE(), table=<customer>), schema='dev')
dict_keys(['dev.customer', 'dev.geo_data'])


In [11]:
# Print the keys (column names) of the 'customer' table
print(customer.columns.keys())

['customerid', 'firstname', 'lastname', 'dateofbirth', 'gender', 'email', 'phonenumber', 'generateddate']


In [12]:
# Construct a SQL SELECT query to retrieve data from the 'customer' table
query = customer.select()  
# Create a ResultProxy by executing the query
exe = conn.execute(query)
# Fetch the first 60 rows of the result
result = exe.fetchmany(60)
# Convert the fetched data into a pandas DataFrame
customer_df = pd.DataFrame(result)
# Print the first 10 rows of the DataFrame
print(customer_df.head(10))
# Display summary statistics for the DataFrame
customer_df.describe()


   customerid firstname  lastname dateofbirth gender                          email   phonenumber generateddate
0           1      John       Doe  1980-05-15      M           john.doe@example.com  123-456-7890    2024-03-25
1           2      Jane     Smith  1992-10-20      F         jane.smith@example.com  987-654-3210    2024-03-25
2           3     David   Johnson  1975-08-12      M      david.johnson@example.com  555-555-5555    2024-03-25
3           4     Sarah  Williams  1988-03-25      F     sarah.williams@example.com  999-999-9999    2024-03-25
4           5   Michael     Brown  1982-11-30      M      michael.brown@example.com  777-777-7777    2024-03-25
5           6     Emily     Jones  1995-06-10      F        emily.jones@example.com  888-888-8888    2024-03-25
6           7    Robert    Garcia  1970-02-18      M      robert.garcia@example.com  222-222-2222    2024-03-25
7           8  Jennifer  Martinez  1986-09-05      F  jennifer.martinez@example.com  333-333-3333    202

,customerid
count,10.00000
mean,5.50000
std,3.02765
min,1.00000
25%,3.25000
50%,5.50000
75%,7.75000
max,10.00000


In [13]:
# Construct a SQL query to select all columns from the 'customer' table where gender is 'M'
query2 = "select * from dev.customer where gender = 'M'"
# Execute the SQL query and store the result
result1 = conn.execute(query2)
# Fetch all rows of the result and convert them into a pandas DataFrame
male_cust_df = pd.DataFrame(result1.fetchall())
# Print the first few rows of the DataFrame
male_cust_df.head()


,customerid,firstname,lastname,dateofbirth,gender,email,phonenumber,generateddate
0,1,John,Doe,1980-05-15,M,john.doe@example.com,123-456-7890,2024-03-25
1,3,David,Johnson,1975-08-12,M,david.johnson@example.com,555-555-5555,2024-03-25
2,5,Michael,Brown,1982-11-30,M,michael.brown@example.com,777-777-7777,2024-03-25
3,7,Robert,Garcia,1970-02-18,M,robert.garcia@example.com,222-222-2222,2024-03-25
4,9,William,Lee,1990-04-17,M,william.lee@example.com,444-444-4444,2024-03-25


In [14]:
# Define database table and schema names
db_table = 'geo_data'
db_schema = 'dev'

# Read geographical data from CSV file into a pandas DataFrame
geosp_data = pd.read_csv("geographical_data.csv", sep=',', skipinitialspace=True)

# Display the first few rows of the DataFrame
geosp_data.head()

try:
    # Write geographical data to PostgreSQL database table
    geosp_data.to_sql(db_table, con=engine, schema=db_schema, index=False, if_exists='replace')
    print("Data successfully written to the database.")
except Exception as e:
    # Print error message if writing data fails
    print("Error:", e)

Data successfully written to the database.
